In [1]:
import sys
sys.path.append("../..")
import numpy as np
import matplotlib.pyplot as plt
from pyfusion import Fusion, AnalogTwo
%matplotlib qt

In [639]:
def load_bitstream_to_flash(cmd_if:serial.Serial, data_if:serial.Serial, path:str, partition_index=0):
    print(f"CMD if = {cmd_if.port}  DATA if = {data_if.port}")
    with open(path, 'rb') as bs:
        data=bs.read()#[26:]
        bs_size=len(data)
        cmd = bytearray(5)
        print(f"Writing bitstream to flash, size={bs_size}")
        cmd[0]=4
        cmd[1]=0
        cmd[2]=0
        cmd[3]=0
        cmd[4]=partition_index&255
        print(cmd)
        cmd_if.write(cmd)
        data_if.write(data)
        remaining = int(1024*1024/8 - bs_size)
        print(f"Stuff with {remaining} x 0xff")
        data_if.write(bytearray([255]* remaining ))

In [6]:
analog_two = AnalogTwo(bitstream_path='../../rtl/top/analog_two/build/top.bit')
analog_two.open()

CMD if = /dev/ttyACM0  DATA if = /dev/ttyACM1
Loading bitstream, size=176582
bytearray(b'\x01\x00\x02\xb1\xc6')


In [7]:
v=analog_two.get_samples(count=200000)

In [4]:
v[0]

array([3920, 3848, 3936, ..., 5148, 5252, 5212], dtype=int16)

In [ ]:
fig,ax= plt.subplots(4, 1)
for a,c in zip(ax,v):
    a.plot(c)

In [8]:
fig,ax= plt.subplots(4, 1)
for a,c in zip(ax,v):
    fft = np.fft.fft(c-np.mean(c))
    freq = np.fft.fftfreq(len(fft), d=1./2e5)[0:len(fft)//2]
    fft = np.abs(fft[0:len(fft)//2])
    a.plot(freq, fft)
    a.semilogx()

In [11]:
fig,ax= plt.subplots(2, 1)
ax[0].specgram(v[0])
ax[1].specgram(v[1])

(array([[6.03514508e+02, 6.05915471e+02, 6.13919210e+02, ...,
         6.01234100e+02, 5.95710646e+02, 6.32388836e+02],
        [2.98690088e+02, 3.06725983e+02, 3.21396791e+02, ...,
         3.05374126e+02, 3.00394290e+02, 3.29664479e+02],
        [5.13665234e-02, 1.94498141e-01, 5.85760126e-01, ...,
         7.08162902e-02, 1.55215933e-02, 4.39257423e-01],
        ...,
        [1.13550166e+00, 2.45563742e+00, 8.75568295e-02, ...,
         2.21877069e+00, 3.69421836e-01, 3.15577546e-01],
        [5.49881905e-02, 1.62608963e-01, 7.68790083e-01, ...,
         3.49518661e+00, 9.08496374e-02, 8.75249011e-02],
        [5.65592309e-01, 1.13763397e+00, 1.50554127e+00, ...,
         1.21610795e+00, 1.48735485e-02, 6.62626438e-04]]),
 array([0.       , 0.0078125, 0.015625 , 0.0234375, 0.03125  , 0.0390625,
        0.046875 , 0.0546875, 0.0625   , 0.0703125, 0.078125 , 0.0859375,
        0.09375  , 0.1015625, 0.109375 , 0.1171875, 0.125    , 0.1328125,
        0.140625 , 0.1484375, 0.15625  , 0.

In [627]:
d=serial.Serial(find_device("DATA"))
d.flushInput()
v=d.read(4096*128)

In [365]:
%timeit d.read(20_000_000)

4.9 s ± 2.03 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
np.where(np.array(list(v), dtype=np.uint8)==0xff)

In [142]:
print(v[4080:4110])

b'\xed\x80\x00\x00\xee\x80\x00\x00\xef\x80\x00\x00\xff\x00\xff\x00\x00\xff\x00\xff\xc9f\x00\x00\xcaf\x00\x00\xcbf'


In [497]:
plt.figure()
data = np.diff(np.frombuffer(v, dtype=np.uint32))
data[data>1e7]=0
plt.plot(data)

In [560]:
values = np.frombuffer(v, dtype=np.int16).copy()

In [5]:
for value,svalue in zip(v[0].astype(np.uint16),v[0]):
    if value==0xf0f0:
        print("================")
    print(f"{np.binary_repr(value, width=16)}\t{svalue}\t{hex(value)}")


0000111101010000	3920	0xf50
0000111100001000	3848	0xf08
0000111101100000	3936	0xf60
0000111101001100	3916	0xf4c
0000111101011100	3932	0xf5c
0000111111000100	4036	0xfc4
0000111110100000	4000	0xfa0
0000111101000100	3908	0xf44
0000111111001100	4044	0xfcc
0000111111110000	4080	0xff0
0000111110010100	3988	0xf94
0000111111100100	4068	0xfe4
0001000000001100	4108	0x100c
0000111111110000	4080	0xff0
0001000000011100	4124	0x101c
0001000001000000	4160	0x1040
0001000000001100	4108	0x100c
0001000000101000	4136	0x1028
0001000001110100	4212	0x1074
0001000000111100	4156	0x103c
0001000001011100	4188	0x105c
0001000001111100	4220	0x107c
0001000001001000	4168	0x1048
0001000011000000	4288	0x10c0
0001000011100100	4324	0x10e4
0001000010001000	4232	0x1088
0001000011000000	4288	0x10c0
0001000100100000	4384	0x1120
0001000011100000	4320	0x10e0
0001000011111100	4348	0x10fc
0001000100110100	4404	0x1134
0001000011101100	4332	0x10ec
0001000100111000	4408	0x1138
0001000101011100	4444	0x115c
0001000101010000	4432	0x115

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [392]:
plt.figure()
plt.plot(np.gradient(values))

In [628]:
values = np.frombuffer(v, dtype=np.int16).copy()[6000:]
for i in range(10):
    if values[i]==-3856:
        values = values[i:].copy()
plt.figure()
for i in range(2,6):
    plt.plot(values[i::6])

In [606]:
plt.figure()
plt.psd(values[3::6].copy(), Fs=2e5, detrend='mean', scale_by_freq=True, NFFT=4096)
#plt.psd(values[3::6].copy(), Fs=2e5, detrend='mean', scale_by_freq=True, NFFT=4096)
#plt.psd(values[4::6].copy(), Fs=2e5, detrend='mean', scale_by_freq=True, NFFT=4096)
#plt.psd(values[5::6].copy(), Fs=2e5, detrend='mean', scale_by_freq=True, NFFT=4096)

(array([4.33121807e-06, 2.37783529e-05, 3.86303005e-05, ...,
        5.80575366e-05, 8.68493859e-05, 3.77967837e-05]),
 array([0.00000000e+00, 4.88281250e+01, 9.76562500e+01, ...,
        9.99023437e+04, 9.99511719e+04, 1.00000000e+05]))

In [589]:
plt.psd?

Signature:
plt.psd(
    x: 'ArrayLike',
    NFFT: 'int | None' = None,
    Fs: 'float | None' = None,
    Fc: 'int | None' = None,
    detrend: "Literal['none', 'mean', 'linear'] | Callable[[ArrayLike], ArrayLike] | None" = None,
    window: 'Callable[[ArrayLike], ArrayLike] | ArrayLike | None' = None,
    noverlap: 'int | None' = None,
    pad_to: 'int | None' = None,
    sides: "Literal['default', 'onesided', 'twosided'] | None" = None,
    scale_by_freq: 'bool | None' = None,
    return_line: 'bool | None' = None,
    *,
    data=None,
    **kwargs,
) -> 'tuple[np.ndarray, np.ndarray] | tuple[np.ndarray, np.ndarray, Line2D]'
Docstring:
Plot the power spectral density.

The power spectral density :math:`P_{xx}` by Welch's average
periodogram method.  The vector *x* is divided into *NFFT* length
segments.  Each segment is detrended by function *detrend* and
windowed by function *window*.  *noverlap* gives the length of
the overlap between segments.  The :math:`|\mathrm{fft}(i)|^2`
of 

In [549]:

for i in range(len(v)):
    if v[i]==0xFF and v[i+1] == 0 and v[i+2]== 0xFF and v[i+3]==0:
        print(i)